# TensorFlow/MNISTによる手書き数字認識システムの構築

---

公開中の "OpenHPC-v2" テンプレートを使うと、クラウド上に GPU ノードクラスタを構築することができます。また、機械学習フレームワークである TensorFlow や Pytorch の環境もサポートしています。

機械学習を使ったアプリケーション環境では、アプリケーションの実行環境では推論のみ行い、学習は別のシステムで行う場合が多くあります。これは、必要な計算能力やメモリ容量が「学習 >> 推論」であることや、秘密事項がある学習データをユーザーがアクセスする環境に置きたくないなどの理由によるものです。

本ハンズオンでは、OpenHPC-v2 で構築した TensorFlow 環境を学習システムとした、手書き数字認識アプリケーション MNIST のアプリケーション実行環境を構築します。この学習環境の構築には時間を要するため、あらかじめ講師の学習環境で学習したニューラルネットのモデルと重みを使います。ハンズオンご参加の皆様には、このテンプレートでアプリ環境を構築していただきます。

本テンプレートの流れ
1. 構築環境情報の入力
1. VCPの初期化
1. CPUのみのVCノードの作成
1. TensorFlow環境の構築
1. MNISTによる手書き数字認識
1. 環境の削除

## 構築環境情報の入力
認識環境の構築情報を入力します。必要に応じ、下記の情報を修正してください。

In [ ]:
# TensorFlow環境のユーザー名
user = 'user00'

###################################################
### ハンズオンでは以下のパラメタを変更しないでください。###
###################################################

# unitgroup名
ugroup_name = 'TfCpu'

# プロバイダ
vc_provider = 'aws'

# フレーバー
vcnode_flavor = 'small'

## VCP の初期化
VCP を利用するために必要なアクセストークンを入力し、VCP SDK を初期化します。

### アクセストークンの入力

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

### VCP の初期化
VCP を初期化します。エラーになった場合、この章のセルを `unfreeze` してから、もう一度アクセストークンを入力してください。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

## CPUのみのVCノードの作成
クラウド上にCPUのみのインスタンスをVCノードとして作成します。
* インスタンス：   
* base コンテナ： ubuntu 16.04

### VCノードのspecを指定する
GPU環境で学習した重みを利用して、CPUのみで推論（本テンプレートの場合は手書き数字認識）するのに十分な性能・容量のノードspecを指定します。

In [ ]:
# UnitGroup の作成
unit_group = vcp.create_ugroup(ugroup_name)

# VCノード spec
spec = vcp.get_spec(vc_provider, vcnode_flavor)

# base コンテナ
spec.image = "vcp/base:1.6.2-ubuntu"

# ssh keyfiles
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')
spec.set_ssh_pubkey(ssh_public_key)

# ssh オプション
ssh_opts = f"-i {ssh_private_key} -l root"

### Unitの作成とVCノードの起動
Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。処理が完了するまで1分半～2分程度かかります。

In [ ]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit('tf-node', spec)

### 疎通確認
まず、sshのknown_hostsの設定を行います。その後、VCノードに対して`uname -a`を実行し、`ubuntu`が起動していることを確認します。`ubuntu`が起動していない場合は、`spec.image` に誤りがあります。本テンプレート下部にある「環境の削除」を実行、`spec.image`を修正、全てのセルを`unfreeze`してから、最初から再実行してください。

In [ ]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す
print(ip_address)

# ssh 設定
!touch ~/.ssh/known_hosts
!sleep 1
!ssh-keygen -R {ip_address}    # ~/.ssh/known_hosts から古いホストキーを削除する
!sleep 1
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts    # ホストキーの登録

# システムの確認
!ssh {ssh_opts} {ip_address} uname -a

## TensorFlow環境の構築
VCノード上に、TensorFlowのインストールページ(https://www.tensorflow.org/install) で紹介されているコンテナイメージを使用して環境を構築します。

### ユーザー登録
作成した Unit に TensorFlow コンテナを実行するユーザーを登録します。便宜上このテンプレートでは、TensorFlow 環境のユーザーのsshの鍵として、この JupyterNotebook 環境の鍵を使用します。

In [ ]:
# ユーザー追加
!ssh {ssh_opts} {ip_address} 'adduser --disabled-login --gecos "" {user}'
!ssh {ssh_opts} {ip_address} usermod -aG 'docker' {user}

# ssh 公開鍵設定
!ssh {ssh_opts} {ip_address} mkdir -m 700 /home/{user}/.ssh
!scp -i {ssh_private_key} {ssh_public_key} root@{ip_address}:/home/{user}/.ssh/authorized_keys
!ssh {ssh_opts} {ip_address} chmod 600 /home/{user}/.ssh/authorized_keys
!ssh {ssh_opts} {ip_address} chown -R {user}:{user} /home/{user}/.ssh/

# ssh 疎通確認
ssh_opts_user = f"-i {ssh_private_key}"
target = f"{user}@{ip_address}"
!ssh {ssh_opts_user} {target} id

### TensorFlow コンテナイメージの作成と実行
ユーザーを追加した TensorFlow コンテナイメージを作成し実行します。

In [ ]:
# 最新のTensorFlowコンテナイメージのダウンロード
docker_image = f'tensorflow/tensorflow:latest'
!ssh {ssh_opts_user} {target} docker pull {docker_image}
# !ssh {ssh_opts} {ip_address} docker info

# ユーザーを追加したコンテナイメージの作成
from pathlib import Path
from tempfile import TemporaryDirectory

!ssh {ssh_opts_user} {target} mkdir -p tensorflow-img
out = !ssh {ssh_opts_user} {target} id -u
uid = out[0]
with TemporaryDirectory() as workdir:
    dockerfile = Path(workdir) / 'Dockerfile'
    with dockerfile.open(mode='w') as f:
        f.write(f'''
FROM {docker_image}

ARG USER={user}
RUN useradd -m ${{USER}} -u {uid}
WORKDIR /home/${{USER}}
USER {user}
''')
    !cat {dockerfile}
    !scp {ssh_opts_user} {dockerfile} {target}:tensorflow-img

!ssh {ssh_opts_user} {target} docker build -t tensorflow-{user} tensorflow-img

# コンテナの実行
!ssh {ssh_opts_user} {target} chmod 755 .
!ssh {ssh_opts_user} {target} docker run -td -v /home/{user}:/home/{user} --rm --ipc=host --net=host --name tensorflow-{user} tensorflow-{user}
!ssh {ssh_opts_user} {target} docker ps

以上で TensorFlow 環境の構築は完了です。

## MNIST による手書き数字の認識
機械学習アプリケーションとして、MNISTによる手書き数字認識を実行します。本章と次章を入れ替えれば、TensorFlowを使う別の機械学習アプリケーションの環境も構築できます。

通常、データー入力と認識は同じシステムで実行することが多いですが、ハンズオン環境の都合上、手書き数字の入力はこの JupyterNotebook 環境で、認識(MNIST）は上で構築した TensorFlow 環境で実行します。入力データはファイルとして転送します。

TensorFlow 環境での動作は、JupyterNotebook から事前に動作を定義したスクリプトを転送しておき、必要になった時点で JupyterNotebook から ssh で起動します。スクリプトは`402/scripts`に格納してあります。

### 実行環境の準備

#### フリーハンド入力の準備
フリーハンド入力機能の定義（プログラム）です。フリーハンド入力は、このJupyterNotebook上で実行します。

In [ ]:
import numpy as np
from IPython.core.display import HTML
import base64
from io import BytesIO
from PIL import Image
from PIL import ImageOps
import matplotlib.pyplot as plt

# フリーハンド入力環境定義
drawpad = """
<canvas id="canvas" height="280px" width="280px" style="border: 1px solid black;"></canvas>
<p>
    <button id="clear">Clear</button>
    <button id="save">Save</button>
</p>
<p id="mesg"></p>

<script type="text/javascript">
    // ====== variables ====== //
    var kernel = IPython.notebook.kernel;
    var btn_clear = document.getElementById("clear");
    var btn_save = document.getElementById("save");
    var canvas = document.getElementById("canvas");
    var context = canvas.getContext("2d");

    var isDrawing = false;
    var x = 0;
    var y = 0;

    // ====== drawing ====== //
    // cf. https://developer.mozilla.org/en-US/docs/Web/API/Element/mousemove_event#examples
    canvas.addEventListener("mousedown", function(e){
        x = e.offsetX;
        y = e.offsetY;
        isDrawing = true;
    });
    
    canvas.addEventListener("mousemove", function(e){
        if (isDrawing === true) {
            drawLine(context, x, y, e.offsetX, e.offsetY);
            x = e.offsetX;
            y = e.offsetY;
        }
    });
    
    canvas.addEventListener("mouseup", function(e){
        if (isDrawing === true) {
            drawLine(context, x, y, e.offsetX, e.offsetY);
            x = 0;
            y = 0;
            isDrawing = false;
        }
    });
    
    canvas.addEventListener("mouseout", function(){
        x = 0;
        y = 0;
        isDrawing = false;
    });

    function drawLine(context, x1, y1, x2, y2) {
        context.beginPath();
        context.strokeStyle = 'black';
        context.lineWidth = 14;
        context.moveTo(x1, y1);
        context.lineTo(x2, y2);
        context.lineCap = "round";
        context.stroke();
    };


    // ====== button ====== //
    btn_clear.addEventListener("click", function(){
        context.clearRect(0, 0, canvas.width, canvas.height);
        mesg.textContent = "";
    });
    
    btn_save.addEventListener("click", function(){
        var img = 'base64_img';
        kernel.execute(img + " = '" + canvas.toDataURL() + "'");
        mesg.textContent = "image saved";
    });
   
</script>
"""

#### MNIST実行の準備
クラウド上に構築した TensorFlow 環境で、MNIST による認識を実行するための準備です。JupyterNotebook から ssh で、TensorFlow 環境を操作します。

MNIST による認識実行で必要なモジュールの TensorFlow 環境へのインストール、ファイル名とパスの設定、VCノード上の作業ディレクトリの作成、VCノード上の作業実行スクリプトのVCノードへの転送を行います。

In [ ]:
# 必要なモジュールのインストール
!ssh {ssh_opts_user} {target} docker exec -t -u {user} tensorflow-{user} pip install pillow
!ssh {ssh_opts_user} {target} docker exec -t -u {user} tensorflow-{user} pip install matplotlib

# ファイル名、パスの設定
scripts = '402/scripts'         # VCノードで実行するスクリプトを格納しているディレクトリ（Jupyter上）
model_dir  = '402/data'         # 学習済みモデル・重みファイルを格納しているディレクトリ（Jupyter上）
model_file = 'saved_model.tgz'  # 学習済みモデル・重みファイル名（Jupyter、VCノード共通）
work_dir = 'tensorflow'         # VCノード上の作業ディレクトリ名（VCノードのTensorFlowユーザホーム上）
img_file = f'{model_dir}/base64_img.b64'  # フリーハンド入力した数字を格納するイメージファイル名（Jupyter上）

# VCノード上の作業ディレクトリの作成
!ssh {ssh_opts_user} {target} mkdir -p {work_dir}

# VCノードで実行するスクリプトの転送
!scp {ssh_opts_user} -qp {scripts}/*.py {target}:{work_dir}

#### MNISTのモデル・重みの準備
GPUを持つ学習システムで学習したモデルと重みを、構築したTensorFlow環境にアップロードし展開します。

In [ ]:
# GPU環境で学習したモデルと重みのアップロードと展開
!ssh {ssh_opts_user} {target} rm -rf {work_dir}/saved_model
!scp {ssh_opts_user} -qp {model_dir}/{model_file} {target}:{work_dir}
!ssh {ssh_opts_user} {target} '(cd {work_dir}; tar zxpf {model_file})'

In [ ]:
# 実はTensorFlowのチュートリアル(初心者のための TensorFlow 2.0 入門)で紹介されているMNISTは、GPUを使わなく
# ても1分以内で学習できます。後ほどコメントを外して試してみてください。
# !ssh {ssh_opts_user} {target} docker exec -t -u {user} -w /home/{user}/{work_dir} tensorflow-{user} python3 download_mnist.py
# !ssh {ssh_opts_user} {target} docker exec -t -u {user} -w /home/{user}/{work_dir} tensorflow-{user} python3 mnist_train.py

### アプリケーションの実行： フリーハンド数字入力と認識
数字（一桁の'0-9'）をフリーハンドで入力します。マウスをクリックしながら描画してください。描画が終わったら、`Save`ボタンをクリックし、２つ目のセルを実行してください。入力したイメージをVCノードのTensorFlow環境に転送し認識します。

再実行：
この入力環境は２つ目のセルを実行した後も機能しています。`Clear`で描画エリアをクリア後、再度入力し`Save`をクリックしてください。認識は２つ目のセルを`unfreeze`してから再実行します。

In [ ]:
# フリーハンド数字入力、base64形式イメージ化
HTML(drawpad)

In [ ]:
# イメージをファイルに保存
with open(img_file, "w") as f1:
    f1.write(base64_img)
    
# VCノードへ転送
!scp {ssh_opts_user} -qp {img_file} {target}:{work_dir}

# 確認のため MNIST の入力サイズ(28x28)に変換して、拡大表示
img = Image.open(BytesIO(base64.b64decode(base64_img.split(",")[-1]))).resize((28,28))
plt.imshow(np.asarray(img))
plt.show()

# MNIST による認識実行
!ssh {ssh_opts_user} {target} docker exec -t -u {user} -w /home/{user}/{work_dir} tensorflow-{user} python3 mnist_predict.py

## 誤答が多い？


学習時のテストでは97%以上の精度が出ているのに、フリーハンド入力の認識では誤りが多いと思います。

### 一般的な機械学習認識システムでの対応
アプリ運用中に誤答が多くなると、誤答データ（イメージ）と正解ラベルを学習システムに戻して追加学習し、この重みで認識システムを更新します。

学習システムはオンデマンドにあれば良いので、本サービスの様にオンデマンドに起動するシステムがあると便利です。

### なぜ、誤答が多い？
このテンプレートのMNISTのモデルは、TensorFlowホームページの「初心者のための TensorFlow 2.0 入門」を基にしています。モデルは、フラットな４層ニューラルネットです（{scripts}/mnist_train.py 参照）。

一般に、フラットなモデルでは、学習データにない筆者の手書き文字を認識するのは難しいと考えられます。学習時のテストで97%以上の高い精度を示したのは、テストデータの中に学習データと同じ筆者のデータが含まれているためと思われます。

学習データにない筆者の手書き認識文字では、畳み込みニューラルネットワーク（CNN）がよく使われます。フラットなネットワークとの違いは、

* CNN: 畳み込み層はイメージまたは前層の出力の小領域の部分特徴を認識します。次の畳み込み層では前層の部分特徴の組み合わせをさらに部分特徴として認識します（結果として、前層より入力層の大きな部分を見ることになります）。これを積み重ねることにより、最終的に全体を認識します
* フラット: 各層でイメージまたは前層の出力全体を認識します

全体しか見ない構造では、変形や位置ずれで誤認識しやすくなります。

### どうする？

In [ ]:
# そこで、CNNで学習した重みも用意してあります。興味のある方は試してみてください。
# モデル構成は、{scripts}/mnist_train_cnn.py を参照してください。

# CNNで学習済みモデル・重みファイル名
model_file = 'saved_model2.tgz'

# GPU環境で学習したモデルと重みのダウンロードと展開
!ssh {ssh_opts_user} {target} rm -rf {work_dir}/saved_model
!scp {ssh_opts_user} -qp {model_dir}/{model_file} {target}:{work_dir}
!ssh {ssh_opts_user} {target} '(cd {work_dir}; tar zxpf {model_file})'

# あとはフリーハンド入力のセルに戻って試してみてください。

In [ ]:
# 実は、CPUだけのVCノードでも学習可能です。8分程度必要です。興味があれば試してみてください。
# !ssh {ssh_opts_user} {target} docker exec -t -u {user} -w /home/{user}/{work_dir} tensorflow-{user} python3 download_mnist.py
# !ssh {ssh_opts_user} {target} docker exec -t -u {user} -w /home/{user}/{work_dir} tensorflow-{user} python3 mnist_train_cnn.py

### CNNを試した方へ

CNNを使っても、あまり認識率が上がらないと思います。

原因の一つとして、モデルの層が少ないことが考えられます。今回使用したCNNは４層です（CNNは最初の2層のみ）。ニューラルネットは層が多いほど複雑な動作ができます。例えば、AND や OR は中間層のない１層で学習できますが、XOR では少なくとも１つの中間層が必要です（２層モデル）。認識精度の高いニューラルネットでは１００層を超えます。

このテンプレートではここまでとしますが、興味ある方は {scripts}/mnist_train_cnn.py を編集して試してみてください。（この環境では学習時間がかかり過ぎますが...）

## 環境の削除
TensorFlowコンテナを停止し、全てのリソースを削除します。

### TensorFlowコンテナの停止

In [ ]:
# TensorFlowコンテナの停止
!ssh {ssh_opts_user} {target} docker stop tensorflow-{user}

### リソースの削除
ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

> 全てのリソースの削除には 4～5分程度かかります。

In [ ]:
unit_group.cleanup()

削除後の状態の確認。

In [ ]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

In [ ]:
# UnitGroup強制削除
# UnitGroup作成後、エラーが発生するなど強制的に削除する必要が生じた場合のみ、コメントを外して利用します。
# ugroup = vcp.get_ugroup('TfCpu')
# ugroup.cleanup()